In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style = 'whitegrid')
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.linear_model import LinearRegression
import scipy.stats as stats
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from scipy.stats import randint as sp_randint, uniform as sp_uniform
import lightgbm as lgb
from xgboost import XGBRFRegressor, XGBRegressor
from catboost import CatBoostRegressor
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.decomposition import PCA

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
train = pd.read_csv('train_dummy_final.csv')
train.head()

,Year,Sales,Events,Federal_Holidays,Nominal_GDP,Real_GDP,CPI,Unemp_Rate,Rate_CreditCard,Rate_PersonalLoan,Wages,Cotton_Price,Change,Upland_Planted,Upland_Harvested,Yield_Harvested,Production,Mill_Use,Exports,Temp_High,Temp_Avg,Temp_Low,Dew_High,Dew_Avg,Dew_Low,Humidity_High,Humidity_Avg,Humidity_Low,Sea_High,Sea_Avg,Sea_Low,Visibility_High,Visibility_Avg,Visibility_Low,Wind_Low,Wind_Avg,Wind_High,Precip,Fog,"Fog , Rain","Fog , Rain , Snow","Fog , Snow",Normal,Rain,"Rain , Snow",Snow,Thunderstorm,Month_2,Month_3,Month_4,Month_5,Month_6,Month_7,Month_8,Month_9,Month_10,Month_11,Month_12,ProductCategory_OtherClothing,ProductCategory_WomenClothing
0,-5,1755.0,0.0,3.0,14421.752895,14407.053343,233.402,7.8,12.03,11.44,22.05,0.5770,4.02,9.296,7.559,799,12.589,4.17,11.55,0.741935,-2.096774,-5.322581,-6.096774,-9.903226,-14.000000,71.774194,57.354839,42.322581,1021.290323,1015.967742,1010.516129,16.0,14.000000,11.451613,27.000000,12.000000,39.322581,2.445806,0.0,0.0,2.0,2.0,21.0,3.0,0.0,3.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,-5,524.0,0.0,3.0,14421.752895,14407.053343,233.402,7.8,12.03,11.44,22.05,0.5770,4.02,9.296,7.559,799,12.589,4.17,11.55,0.741935,-2.096774,-5.322581,-6.096774,-9.903226,-14.000000,71.774194,57.354839,42.322581,1021.290323,1015.967742,1010.516129,16.0,14.000000,11.451613,27.000000,12.000000,39.322581,2.445806,0.0,0.0,2.0,2.0,21.0,3.0,0.0,3.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,-5,936.0,0.0,3.0,14421.752895,14407.053343,233.402,7.8,12.03,11.44,22.05,0.5770,4.02,9.296,7.559,799,12.589,4.17,11.55,0.741935,-2.096774,-5.322581,-6.096774,-9.903226,-14.000000,71.774194,57.354839,42.322581,1021.290323,1015.967742,1010.516129,16.0,14.000000,11.451613,27.000000,12.000000,39.322581,2.445806,0.0,0.0,2.0,2.0,21.0,3.0,0.0,3.0,0.0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,-5,1729.0,1.0,1.0,14389.200466,14366.176571,234.663,8.3,12.97,11.05,22.22,0.5521,-4.32,9.296,7.559,799,12.589,3.87,11.10,6.964286,2.785714,-1.642857,-2.464286,-6.392857,-10.571429,69.535714,54.000000,38.107143,1021.714286,1017.071429,1012.464286,16.0,15.178571,13.285714,27.928571,12.214286,41.642857,0.782500,0.0,0.0,0.0,1.0,22.0,4.0,1.0,0.0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,1
4,-5,496.0,1.0,1.0,14389.200466,14366.176571,234.663,8.3,12.97,11.05,22.22,0.5521,-4.32,9.296,7.559,799,12.589,3.87,11.10,6.964286,2.785714,-1.642857,-2.464286,-6.392857,-10.571429,69.535714,54.000000,38.107143,1021.714286,1017.071429,1012.464286,16.0,15.178571,13.285714,27.928571,12.214286,41.642857,0.782500,0.0,0.0,0.0,1.0,22.0,4.0,1.0,0.0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [5]:
test = pd.read_csv('test_dummy_final.csv')
test.head()

,Year,Events,Federal_Holidays,Nominal_GDP,Real_GDP,CPI,Unemp_Rate,Rate_CreditCard,Rate_PersonalLoan,Wages,Cotton_Price,Change,Upland_Planted,Upland_Harvested,Yield_Harvested,Production,Mill_Use,Exports,Temp_High,Temp_Avg,Temp_Low,Dew_High,Dew_Avg,Dew_Low,Humidity_High,Humidity_Avg,Humidity_Low,Sea_High,Sea_Avg,Sea_Low,Visibility_High,Visibility_Avg,Visibility_Low,Wind_Low,Wind_Avg,Wind_High,Precip,Fog,"Fog , Rain","Fog , Rain , Snow","Fog , Snow",Normal,Rain,"Rain , Snow",Snow,Thunderstorm,Month_2,Month_3,Month_4,Month_5,Month_6,Month_7,Month_8,Month_9,Month_10,Month_11,Month_12,ProductCategory_OtherClothing,ProductCategory_WomenClothing
0,0,0.0,3.0,16955.608626,15705.972667,259.596,6.6,11.85,10.22,24.35,0.9096,3.97,10.206,7.465,807,12.551,3.58,9.75,1.733333,-1.766667,-5.700000,-5.166667,-9.400000,-13.833333,75.033333,59.433333,43.900000,1025.033333,1019.200000,1012.233333,15.90,13.866667,10.633333,22.700000,10.10,38.866667,2.000333,0.0,3.0,0.0,1.0,20.0,2.0,1.0,3.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,0,0.0,3.0,16955.608626,15705.972667,259.596,6.6,11.85,10.22,24.35,0.9096,3.97,10.206,7.465,807,12.551,3.58,9.75,1.733333,-1.766667,-5.700000,-5.166667,-9.400000,-13.833333,75.033333,59.433333,43.900000,1025.033333,1019.200000,1012.233333,15.90,13.866667,10.633333,22.700000,10.10,38.866667,2.000333,0.0,3.0,0.0,1.0,20.0,2.0,1.0,3.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0.0,3.0,16955.608626,15705.972667,259.596,6.6,11.85,10.22,24.35,0.9096,3.97,10.206,7.465,807,12.551,3.58,9.75,1.733333,-1.766667,-5.700000,-5.166667,-9.400000,-13.833333,75.033333,59.433333,43.900000,1025.033333,1019.200000,1012.233333,15.90,13.866667,10.633333,22.700000,10.10,38.866667,2.000333,0.0,3.0,0.0,1.0,20.0,2.0,1.0,3.0,0.0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,0,1.0,1.0,17028.586348,15762.598920,259.019,6.7,11.83,10.09,24.58,0.9405,3.40,10.206,7.465,807,12.551,3.58,9.75,3.285714,0.285714,-3.107143,-4.178571,-7.821429,-11.678571,74.892857,58.714286,42.035714,1022.321429,1016.964286,1011.964286,15.75,12.357143,8.535714,25.178571,10.25,41.392857,4.972857,0.0,1.0,4.0,3.0,17.0,2.0,0.0,1.0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,1
4,0,1.0,1.0,17028.586348,15762.598920,259.019,6.7,11.83,10.09,24.58,0.9405,3.40,10.206,7.465,807,12.551,3.58,9.75,3.285714,0.285714,-3.107143,-4.178571,-7.821429,-11.678571,74.892857,58.714286,42.035714,1022.321429,1016.964286,1011.964286,15.75,12.357143,8.535714,25.178571,10.25,41.392857,4.972857,0.0,1.0,4.0,3.0,17.0,2.0,0.0,1.0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [6]:
X_train = train.drop('Sales', 1)
y_train = train['Sales']
X_test = test

In [7]:
y_train_log = np.log(y_train)

In [8]:
ss = StandardScaler()
X_train_scaled = pd.DataFrame(ss.fit_transform(X_train), columns = X_train.columns)
X_test_scaled = pd.DataFrame(ss.transform(X_test), columns = X_test.columns)

In [9]:
X_train_scaled.head()

,Year,Events,Federal_Holidays,Nominal_GDP,Real_GDP,CPI,Unemp_Rate,Rate_CreditCard,Rate_PersonalLoan,Wages,Cotton_Price,Change,Upland_Planted,Upland_Harvested,Yield_Harvested,Production,Mill_Use,Exports,Temp_High,Temp_Avg,Temp_Low,Dew_High,Dew_Avg,Dew_Low,Humidity_High,Humidity_Avg,Humidity_Low,Sea_High,Sea_Avg,Sea_Low,Visibility_High,Visibility_Avg,Visibility_Low,Wind_Low,Wind_Avg,Wind_High,Precip,Fog,"Fog , Rain","Fog , Rain , Snow","Fog , Snow",Normal,Rain,"Rain , Snow",Snow,Thunderstorm,Month_2,Month_3,Month_4,Month_5,Month_6,Month_7,Month_8,Month_9,Month_10,Month_11,Month_12,ProductCategory_OtherClothing,ProductCategory_WomenClothing
0,-1.414214,-1.063410,1.717911,-1.356783,-1.456054,-1.803351,-0.947473,-0.956059,1.799090,-1.520639,-1.087254,0.357995,-0.99312,-1.168809,0.125973,-1.051372,3.978623,-0.109905,-1.818354,-1.799562,-1.778527,-1.749912,-1.751915,-1.750437,-1.227609,-0.934442,-0.570120,0.588624,0.084808,-0.590389,0.756966,0.146405,0.614829,1.091576,1.257299,0.812422,-0.480800,-0.428499,-1.028992,3.780756,1.473935,0.499525,-1.379827,-0.466252,1.586474,-0.130189,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.707107,1.414214
1,-1.414214,-1.063410,1.717911,-1.356783,-1.456054,-1.803351,-0.947473,-0.956059,1.799090,-1.520639,-1.087254,0.357995,-0.99312,-1.168809,0.125973,-1.051372,3.978623,-0.109905,-1.818354,-1.799562,-1.778527,-1.749912,-1.751915,-1.750437,-1.227609,-0.934442,-0.570120,0.588624,0.084808,-0.590389,0.756966,0.146405,0.614829,1.091576,1.257299,0.812422,-0.480800,-0.428499,-1.028992,3.780756,1.473935,0.499525,-1.379827,-0.466252,1.586474,-0.130189,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.707107,-0.707107
2,-1.414214,-1.063410,1.717911,-1.356783,-1.456054,-1.803351,-0.947473,-0.956059,1.799090,-1.520639,-1.087254,0.357995,-0.99312,-1.168809,0.125973,-1.051372,3.978623,-0.109905,-1.818354,-1.799562,-1.778527,-1.749912,-1.751915,-1.750437,-1.227609,-0.934442,-0.570120,0.588624,0.084808,-0.590389,0.756966,0.146405,0.614829,1.091576,1.257299,0.812422,-0.480800,-0.428499,-1.028992,3.780756,1.473935,0.499525,-1.379827,-0.466252,1.586474,-0.130189,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,1.414214,-0.707107
3,-1.414214,0.096674,-0.156174,-1.395944,-1.550379,-1.635694,-0.392313,0.241403,0.748985,-1.260352,-1.151090,-0.664555,-0.99312,-1.168809,0.125973,-1.051372,2.165074,-0.365719,-1.131904,-1.234187,-1.334144,-1.320196,-1.357616,-1.382713,-1.573152,-1.472215,-1.200227,0.714550,0.462549,0.107931,0.756966,1.447747,1.760697,1.374359,1.359193,1.338401,-1.160801,-0.428499,-1.028992,-0.420084,0.512673,0.852131,-1.057689,1.398757,-0.438812,-0.130189,3.316625,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.707107,1.414214
4,-1.414214,0.096674,-0.156174,-1.395944,-1.550379,-1.635694,-0.392313,0.241403,0.748985,-1.260352,-1.151090,-0.664555,-0.99312,-1.168809,0.125973,-1.051372,2.165074,-0.365719,-1.131904,-1.234187,-1.334144,-1.320196,-1.357616,-1.382713,-1.573152,-1.472215,-1.200227,0.714550,0.462549,0.107931,0.756966,1.447747,1.760697,1.374359,1.359193,1.338401,-1.160801,-0.428499,-1.028992,-0.420084,0.512673,0.852131,-1.057689,1.398757,-0.438812,-0.130189,3.316625,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.707107,-0.707107


In [10]:
X_test_scaled.head()

,Year,Events,Federal_Holidays,Nominal_GDP,Real_GDP,CPI,Unemp_Rate,Rate_CreditCard,Rate_PersonalLoan,Wages,Cotton_Price,Change,Upland_Planted,Upland_Harvested,Yield_Harvested,Production,Mill_Use,Exports,Temp_High,Temp_Avg,Temp_Low,Dew_High,Dew_Avg,Dew_Low,Humidity_High,Humidity_Avg,Humidity_Low,Sea_High,Sea_Avg,Sea_Low,Visibility_High,Visibility_Avg,Visibility_Low,Wind_Low,Wind_Avg,Wind_High,Precip,Fog,"Fog , Rain","Fog , Rain , Snow","Fog , Snow",Normal,Rain,"Rain , Snow",Snow,Thunderstorm,Month_2,Month_3,Month_4,Month_5,Month_6,Month_7,Month_8,Month_9,Month_10,Month_11,Month_12,ProductCategory_OtherClothing,ProductCategory_WomenClothing
0,2.12132,-1.063410,1.717911,1.691501,1.541286,1.679290,-2.279856,-1.185360,-1.485853,2.000895,-0.234573,0.351864,-0.504925,-1.246582,0.409856,-1.068776,0.411978,-1.133161,-1.708983,-1.761337,-1.824106,-1.639882,-1.695390,-1.732562,-0.724512,-0.601263,-0.334334,1.700379,1.191060,0.025146,-0.002987,-0.000818,0.103603,-0.217929,0.353840,0.709072,-0.66292,-0.428499,0.685994,-0.420084,0.512673,0.146919,-1.701966,1.398757,1.586474,-0.130189,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.707107,1.414214
1,2.12132,-1.063410,1.717911,1.691501,1.541286,1.679290,-2.279856,-1.185360,-1.485853,2.000895,-0.234573,0.351864,-0.504925,-1.246582,0.409856,-1.068776,0.411978,-1.133161,-1.708983,-1.761337,-1.824106,-1.639882,-1.695390,-1.732562,-0.724512,-0.601263,-0.334334,1.700379,1.191060,0.025146,-0.002987,-0.000818,0.103603,-0.217929,0.353840,0.709072,-0.66292,-0.428499,0.685994,-0.420084,0.512673,0.146919,-1.701966,1.398757,1.586474,-0.130189,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.707107,-0.707107
2,2.12132,-1.063410,1.717911,1.691501,1.541286,1.679290,-2.279856,-1.185360,-1.485853,2.000895,-0.234573,0.351864,-0.504925,-1.246582,0.409856,-1.068776,0.411978,-1.133161,-1.708983,-1.761337,-1.824106,-1.639882,-1.695390,-1.732562,-0.724512,-0.601263,-0.334334,1.700379,1.191060,0.025146,-0.002987,-0.000818,0.103603,-0.217929,0.353840,0.709072,-0.66292,-0.428499,0.685994,-0.420084,0.512673,0.146919,-1.701966,1.398757,1.586474,-0.130189,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,1.414214,-0.707107
3,2.12132,0.096674,-0.156174,1.779294,1.671955,1.602575,-2.168824,-1.210838,-1.835888,2.353048,-0.155355,0.281978,-0.504925,-1.246582,0.409856,-1.068776,0.411978,-1.133161,-1.537724,-1.523678,-1.510979,-1.522993,-1.518079,-1.501457,-0.746197,-0.716525,-0.613000,0.894885,0.425879,-0.071295,-1.142915,-1.667587,-1.206899,0.536885,0.425166,1.281729,0.55232,-0.428499,-0.457330,7.981596,2.435197,-0.910898,-1.701966,-0.466252,0.236283,-0.130189,3.316625,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.707107,1.414214
4,2.12132,0.096674,-0.156174,1.779294,1.671955,1.602575,-2.168824,-1.210838,-1.835888,2.353048,-0.155355,0.281978,-0.504925,-1.246582,0.409856,-1.068776,0.411978,-1.133161,-1.537724,-1.523678,-1.510979,-1.522993,-1.518079,-1.501457,-0.746197,-0.716525,-0.613000,0.894885,0.425879,-0.071295,-1.142915,-1.667587,-1.206899,0.536885,0.425166,1.281729,0.55232,-0.428499,-0.457330,7.981596,2.435197,-0.910898,-1.701966,-0.466252,0.236283,-0.130189,3.316625,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.301511,-0.707107,-0.707107


In [295]:
train.corr()

,Year,Sales,Events,Federal_Holidays,Nominal_GDP,Real_GDP,CPI,Unemp_Rate,Rate_CreditCard,Rate_PersonalLoan,Wages,Cotton_Price,Change,Upland_Planted,Upland_Harvested,Yield_Harvested,Production,Mill_Use,Exports,Temp_High,Temp_Avg,Temp_Low,Dew_High,Dew_Avg,Dew_Low,Humidity_High,Humidity_Avg,Humidity_Low,Sea_High,Sea_Avg,Sea_Low,Visibility_High,Visibility_Avg,Visibility_Low,Wind_Low,Wind_Avg,Wind_High,Precip,Fog,"Fog , Rain","Fog , Rain , Snow","Fog , Snow",Normal,Rain,"Rain , Snow",Snow,Thunderstorm,Month_2,Month_3,Month_4,Month_5,Month_6,Month_7,Month_8,Month_9,Month_10,Month_11,Month_12,ProductCategory_OtherClothing,ProductCategory_WomenClothing
Year,1.000000e+00,0.148636,-8.095284e-18,-9.808290e-18,9.759554e-01,9.652936e-01,9.728776e-01,-8.426894e-01,-7.965902e-01,-7.482481e-01,9.742067e-01,1.486192e-01,-2.249061e-01,5.183026e-01,2.392999e-01,7.485765e-02,2.307609e-01,-2.236315e-01,-1.351299e-01,2.827989e-02,2.846176e-02,3.250016e-02,2.179891e-02,8.020365e-03,1.943640e-03,-8.656168e-02,-1.072168e-01,-1.160109e-01,1.786798e-01,2.018135e-01,2.212183e-01,-3.188161e-01,-7.610502e-02,-1.086415e-01,-2.542553e-01,-2.873278e-01,3.096916e-02,-9.999190e-02,1.928148e-01,-6.737101e-03,1.237684e-01,-1.132858e-01,-8.310999e-03,-4.935373e-02,1.318761e-01,7.160469e-02,2.980959e-18,-1.183502e-17,2.367003e-18,-5.523008e-18,-1.183502e-17,-5.523008e-18,-1.183502e-17,8.679013e-18,-5.523008e-18,-3.945006e-18,-5.523008e-18,6.312009e-18,0.000000e+00,0.000000e+00
Sales,1.486360e-01,1.000000,1.857003e-01,3.868227e-02,1.726492e-01,1.748671e-01,1.659348e-01,-1.271768e-01,-1.204572e-01,-1.360616e-01,1.671478e-01,4.047378e-02,-4.142393e-02,9.456462e-02,5.044371e-02,-2.115728e-02,4.042298e-02,-5.255582e-02,-1.439484e-02,-1.260664e-02,-1.222109e-02,-1.149498e-02,-8.752332e-03,-1.254476e-02,-1.380286e-02,-1.238605e-02,-6.170892e-03,1.667997e-04,6.245844e-02,6.092187e-02,5.416986e-02,-7.260121e-02,-1.830220e-02,-1.130335e-02,-2.306346e-02,-2.809044e-02,3.531660e-02,7.658432e-03,5.831717e-02,-3.077740e-02,2.856517e-02,-3.116430e-02,2.063062e-02,3.196605e-02,4.442882e-02,-5.140152e-02,-1.394901e-02,-7.958057e-02,3.445254e-03,3.065013e-02,3.227911e-02,-1.455195e-02,-3.873406e-02,-1.892494e-02,-1.868974e-02,-4.333790e-03,2.816745e-02,1.841490e-01,-3.346288e-01,9.208460e-01
Events,-8.095284e-18,0.185700,1.000000e+00,1.056852e-01,5.744627e-02,6.886740e-02,9.086650e-03,-3.291718e-02,2.272158e-02,-4.325338e-02,9.727196e-02,9.732661e-02,6.391310e-02,1.371180e-02,-7.159963e-03,-1.658647e-01,-4.448708e-02,-8.739818e-02,1.513139e-02,-4.130518e-01,-4.230662e-01,-4.331784e-01,-4.168906e-01,-4.291172e-01,-4.360285e-01,-2.704603e-01,-2.334660e-01,-1.745553e-01,2.104644e-01,6.864122e-02,-9.752132e-02,-5.658926e-02,-4.484526e-02,4.553704e-02,4.509831e-01,4.260443e-01,5.373465e-01,-9.346397e-03,1.318055e-01,-3.979055e-01,1.624442e-01,1.177097e-01,-1.096487e-01,2.081345e-01,2.614311e-01,-4.894796e-02,-1.384442e-01,2.914820e-02,2.914820e-02,3.789266e-01,2.914820e-02,2.914820e-02,-3.206302e-01,-3.206302e-01,-3.206302e-01,2.914820e-02,2.914820e-02,7.287050e-01,-3.035731e-18,0.000000e+00
Federal_Holidays,-9.808290e-18,0.038682,1.056852e-01,1.000000e+00,4.691908e-02,5.416269e-02,-2.188650e-02,-6.820508e-02,-1.015635e-01,3.707495e-02,1.104326e-01,-2.193216e-02,1.365039e-01,3.395900e-02,1.399594e-02,-7.786348e-02,-6.229100e-03,8.293879e-02,-1.114497e-01,-4.743180e-01,-4.546428e-01,-4.321629e-01,-4.089591e-01,-3.976989e-01,-3.879465e-01,-2.559895e-01,-6.736787e-02,1.206660e-01,3.296533e-01,2.547459e-01,1.149255e-01,-1.732610e-01,-3.927026e-02,1.323102e-01,2.414379e-01,2.804881e-01,2.094682e-01,-1.744405e-01,-3.041833e-02,-1.428458e-01,3.280305e-01,3.953263e-01,1.385872e-01,-3.563597e-01,4.514616e-01,2.161360e-01,1.016605e-01,-4.708816e-02,-3.296171e-01,-3.296171e-01,-4.708816e-02,-3.296171e-01,2.354408e-01,-3.296171e-01,-4.708816e-02,-4.708816e-02,2.354408e-01,5.179698e-01,1.226036e-17,1.226036e-17
Nominal_GDP,9.759554e-01,0.172649,5.744627e-02,4.691908e-02,1.000000e+00,9.948168e-01,9.822

### LR

In [23]:
lr = LinearRegression()
lr.fit(X_train_scaled, y_train_log)
lr.score(X_train_scaled, y_train_log)

0.9879847550940997

### Polynomial Features

In [556]:
poly = PolynomialFeatures(2)
X_train_poly2 = poly.fit_transform(X_train_scaled)
X_test_poly2 = poly.transform(X_test_scaled)

In [557]:
lr = LinearRegression()
lr.fit(X_train_poly2, y_train_log)
lr.score(X_train_poly2, y_train_log)

0.9999158329631967

In [562]:
X_train_poly2.shape

(180, 1830)

In [559]:
poly = PolynomialFeatures(3)
X_train_poly3 = poly.fit_transform(X_train_scaled)
X_test_poly3 = poly.transform(X_test_scaled)

In [560]:
lr = LinearRegression()
lr.fit(X_train_poly3, y_train_log)
lr.score(X_train_poly3, y_train_log)

1.0

In [561]:
X_train_poly3.shape

(180, 37820)

### PCA

In [572]:
pca = PCA(50)

In [573]:
X_train_pca = pca.fit_transform(X_train_poly2)
X_test_pca = pca.transform(X_test_poly2)

In [574]:
pca.explained_variance_ratio_.sum()

0.9251793873585735

In [575]:
lr = LinearRegression()
lr.fit(X_train_pca, y_train_log)
lr.score(X_train_pca, y_train_log)

0.06879664836015364

### Modeling

In [576]:
rfr = RandomForestRegressor(random_state = 0)
cbr = CatBoostRegressor(random_state = 0)
gbr = GradientBoostingRegressor(random_state = 0)
lgbmr = lgb.LGBMRegressor(random_state = 0)
xgbr = XGBRegressor(random_state = 0)
xgbrfr = XGBRFRegressor(random_state = 0)
votreg = VotingRegressor(estimators = [('rfr', rfr), ('cbr', cbr), ('gbr', gbr), ('xgbr', xgbr), ('xgbrfr', xgbrfr)])
votreg2 = VotingRegressor(estimators = [('rfr', rfr), ('gbr', gbr)])

#### RF

In [538]:
rfr.fit(X_train_pca, y_train_log)
rfr.score(X_train_pca, y_train_log)

0.9685707594596914

##### Tuning RF

In [539]:
rfr = RandomForestRegressor(random_state = 0)
params = {'n_estimators' : sp_randint(50, 200),
         'max_features' : sp_randint(1, 59),
         'max_depth' : sp_randint(2, 50),
         'min_samples_leaf' : sp_randint(1, 25),
         'min_samples_split' : sp_randint(2, 25),
         'oob_score' : [False, True]}
rsearch = RandomizedSearchCV(rfr, param_distributions=params, cv = 5, n_jobs = -1, random_state = 0, n_iter = 100)
rsearch.fit(X_train_pca, y_train_log)
rsearch.best_params_

{'max_depth': 29,
 'max_features': 52,
 'min_samples_leaf': 4,
 'min_samples_split': 11,
 'n_estimators': 68,
 'oob_score': True}

#### Tuned RF

In [540]:
rfr = RandomForestRegressor(**rsearch.best_params_, random_state = 0)
rfr.fit(X_train_pca, y_train_log)
rfr.score(X_train_pca, y_train_log)

0.933197489518335

#### CBR

In [541]:
cbr.fit(X_train_pca, y_train_log)
cbr.score(X_train_pca, y_train_log)

Learning rate set to 0.028763
0:	learn: 0.6658609	total: 18.1ms	remaining: 18.1s
1:	learn: 0.6565848	total: 29.1ms	remaining: 14.5s
2:	learn: 0.6457129	total: 43.4ms	remaining: 14.4s
3:	learn: 0.6381090	total: 53.2ms	remaining: 13.3s
4:	learn: 0.6328292	total: 63ms	remaining: 12.5s
5:	learn: 0.6261006	total: 73.6ms	remaining: 12.2s
6:	learn: 0.6187049	total: 85.5ms	remaining: 12.1s
7:	learn: 0.6128191	total: 95.2ms	remaining: 11.8s
8:	learn: 0.6023836	total: 105ms	remaining: 11.6s
9:	learn: 0.5956610	total: 115ms	remaining: 11.4s
10:	learn: 0.5858189	total: 128ms	remaining: 11.5s
11:	learn: 0.5758920	total: 139ms	remaining: 11.5s
12:	learn: 0.5669337	total: 149ms	remaining: 11.3s
13:	learn: 0.5632598	total: 159ms	remaining: 11.2s
14:	learn: 0.5565164	total: 171ms	remaining: 11.2s
15:	learn: 0.5480217	total: 184ms	remaining: 11.3s
16:	learn: 0.5420527	total: 196ms	remaining: 11.3s
17:	learn: 0.5342760	total: 212ms	remaining: 11.6s
18:	learn: 0.5241367	total: 226ms	remaining: 11.6s
19:	l

160:	learn: 0.1357529	total: 2.38s	remaining: 12.4s
161:	learn: 0.1351694	total: 2.41s	remaining: 12.5s
162:	learn: 0.1336273	total: 2.43s	remaining: 12.5s
163:	learn: 0.1328336	total: 2.44s	remaining: 12.4s
164:	learn: 0.1312283	total: 2.46s	remaining: 12.4s
165:	learn: 0.1306318	total: 2.47s	remaining: 12.4s
166:	learn: 0.1294736	total: 2.48s	remaining: 12.4s
167:	learn: 0.1285134	total: 2.5s	remaining: 12.4s
168:	learn: 0.1270726	total: 2.5s	remaining: 12.3s
169:	learn: 0.1261789	total: 2.52s	remaining: 12.3s
170:	learn: 0.1250309	total: 2.54s	remaining: 12.3s
171:	learn: 0.1242820	total: 2.56s	remaining: 12.3s
172:	learn: 0.1237066	total: 2.57s	remaining: 12.3s
173:	learn: 0.1231052	total: 2.58s	remaining: 12.3s
174:	learn: 0.1222375	total: 2.59s	remaining: 12.2s
175:	learn: 0.1213653	total: 2.61s	remaining: 12.2s
176:	learn: 0.1201472	total: 2.62s	remaining: 12.2s
177:	learn: 0.1196019	total: 2.63s	remaining: 12.2s
178:	learn: 0.1188573	total: 2.65s	remaining: 12.1s
179:	learn: 0.

330:	learn: 0.0437382	total: 4.68s	remaining: 9.47s
331:	learn: 0.0434129	total: 4.7s	remaining: 9.46s
332:	learn: 0.0432590	total: 4.71s	remaining: 9.44s
333:	learn: 0.0429314	total: 4.73s	remaining: 9.43s
334:	learn: 0.0426064	total: 4.74s	remaining: 9.4s
335:	learn: 0.0424024	total: 4.75s	remaining: 9.38s
336:	learn: 0.0421563	total: 4.76s	remaining: 9.37s
337:	learn: 0.0418697	total: 4.77s	remaining: 9.34s
338:	learn: 0.0416810	total: 4.78s	remaining: 9.32s
339:	learn: 0.0415458	total: 4.79s	remaining: 9.3s
340:	learn: 0.0412459	total: 4.8s	remaining: 9.28s
341:	learn: 0.0409382	total: 4.81s	remaining: 9.26s
342:	learn: 0.0405856	total: 4.83s	remaining: 9.24s
343:	learn: 0.0403694	total: 4.84s	remaining: 9.23s
344:	learn: 0.0402080	total: 4.85s	remaining: 9.21s
345:	learn: 0.0400699	total: 4.87s	remaining: 9.2s
346:	learn: 0.0399398	total: 4.89s	remaining: 9.2s
347:	learn: 0.0398215	total: 4.9s	remaining: 9.18s
348:	learn: 0.0397600	total: 4.91s	remaining: 9.16s
349:	learn: 0.03969

492:	learn: 0.0212785	total: 6.68s	remaining: 6.87s
493:	learn: 0.0212448	total: 6.69s	remaining: 6.86s
494:	learn: 0.0210567	total: 6.71s	remaining: 6.84s
495:	learn: 0.0208980	total: 6.73s	remaining: 6.84s
496:	learn: 0.0207454	total: 6.74s	remaining: 6.82s
497:	learn: 0.0206920	total: 6.75s	remaining: 6.8s
498:	learn: 0.0206453	total: 6.76s	remaining: 6.79s
499:	learn: 0.0206134	total: 6.78s	remaining: 6.78s
500:	learn: 0.0205788	total: 6.79s	remaining: 6.76s
501:	learn: 0.0205440	total: 6.8s	remaining: 6.75s
502:	learn: 0.0205072	total: 6.81s	remaining: 6.73s
503:	learn: 0.0204862	total: 6.82s	remaining: 6.71s
504:	learn: 0.0203468	total: 6.83s	remaining: 6.7s
505:	learn: 0.0203162	total: 6.85s	remaining: 6.68s
506:	learn: 0.0202182	total: 6.86s	remaining: 6.67s
507:	learn: 0.0201769	total: 6.88s	remaining: 6.66s
508:	learn: 0.0200970	total: 6.89s	remaining: 6.65s
509:	learn: 0.0200371	total: 6.9s	remaining: 6.63s
510:	learn: 0.0200094	total: 6.92s	remaining: 6.62s
511:	learn: 0.01

651:	learn: 0.0115955	total: 9.65s	remaining: 5.15s
652:	learn: 0.0115864	total: 9.69s	remaining: 5.15s
653:	learn: 0.0115739	total: 9.7s	remaining: 5.13s
654:	learn: 0.0115621	total: 9.71s	remaining: 5.12s
655:	learn: 0.0114782	total: 9.74s	remaining: 5.11s
656:	learn: 0.0114655	total: 9.75s	remaining: 5.09s
657:	learn: 0.0114330	total: 9.76s	remaining: 5.07s
658:	learn: 0.0113876	total: 9.77s	remaining: 5.06s
659:	learn: 0.0113008	total: 9.8s	remaining: 5.05s
660:	learn: 0.0112899	total: 9.81s	remaining: 5.03s
661:	learn: 0.0112645	total: 9.83s	remaining: 5.02s
662:	learn: 0.0112005	total: 9.86s	remaining: 5.01s
663:	learn: 0.0111197	total: 9.87s	remaining: 4.99s
664:	learn: 0.0111091	total: 9.88s	remaining: 4.98s
665:	learn: 0.0110810	total: 9.91s	remaining: 4.97s
666:	learn: 0.0110131	total: 9.94s	remaining: 4.96s
667:	learn: 0.0109609	total: 9.95s	remaining: 4.95s
668:	learn: 0.0108816	total: 9.96s	remaining: 4.93s
669:	learn: 0.0108307	total: 9.98s	remaining: 4.92s
670:	learn: 0.

811:	learn: 0.0065460	total: 12.4s	remaining: 2.88s
812:	learn: 0.0065411	total: 12.5s	remaining: 2.86s
813:	learn: 0.0065371	total: 12.5s	remaining: 2.85s
814:	learn: 0.0065167	total: 12.5s	remaining: 2.83s
815:	learn: 0.0065023	total: 12.5s	remaining: 2.82s
816:	learn: 0.0064985	total: 12.5s	remaining: 2.8s
817:	learn: 0.0064530	total: 12.5s	remaining: 2.78s
818:	learn: 0.0064027	total: 12.5s	remaining: 2.77s
819:	learn: 0.0063930	total: 12.5s	remaining: 2.75s
820:	learn: 0.0063777	total: 12.6s	remaining: 2.74s
821:	learn: 0.0063737	total: 12.6s	remaining: 2.72s
822:	learn: 0.0063358	total: 12.6s	remaining: 2.7s
823:	learn: 0.0063236	total: 12.6s	remaining: 2.69s
824:	learn: 0.0063058	total: 12.6s	remaining: 2.67s
825:	learn: 0.0062966	total: 12.6s	remaining: 2.66s
826:	learn: 0.0062851	total: 12.6s	remaining: 2.64s
827:	learn: 0.0062280	total: 12.6s	remaining: 2.63s
828:	learn: 0.0062241	total: 12.7s	remaining: 2.61s
829:	learn: 0.0061838	total: 12.7s	remaining: 2.6s
830:	learn: 0.0

978:	learn: 0.0036051	total: 14.6s	remaining: 314ms
979:	learn: 0.0035918	total: 14.7s	remaining: 300ms
980:	learn: 0.0035883	total: 14.7s	remaining: 284ms
981:	learn: 0.0035699	total: 14.7s	remaining: 270ms
982:	learn: 0.0035677	total: 14.8s	remaining: 255ms
983:	learn: 0.0035396	total: 14.8s	remaining: 240ms
984:	learn: 0.0035220	total: 14.8s	remaining: 225ms
985:	learn: 0.0035202	total: 14.8s	remaining: 211ms
986:	learn: 0.0034930	total: 14.8s	remaining: 196ms
987:	learn: 0.0034773	total: 14.9s	remaining: 181ms
988:	learn: 0.0034740	total: 14.9s	remaining: 166ms
989:	learn: 0.0034717	total: 14.9s	remaining: 151ms
990:	learn: 0.0034513	total: 14.9s	remaining: 136ms
991:	learn: 0.0034458	total: 15s	remaining: 121ms
992:	learn: 0.0034375	total: 15s	remaining: 106ms
993:	learn: 0.0034128	total: 15s	remaining: 90.7ms
994:	learn: 0.0033986	total: 15s	remaining: 75.5ms
995:	learn: 0.0033882	total: 15.1s	remaining: 60.5ms
996:	learn: 0.0033848	total: 15.1s	remaining: 45.4ms
997:	learn: 0.00

0.9999753096901038

#### GBR

In [542]:
gbr.fit(X_train_pca, y_train_log)
gbr.score(X_train_pca, y_train_log)

0.9918343864340027

##### Tuning GBR

In [543]:
gbr = GradientBoostingRegressor(random_state = 0)

params = {'n_estimators' : sp_randint(50, 200),
          'max_features' : sp_randint(1, 59),
          'max_depth' : sp_randint(2, 50),
          'min_samples_leaf' : sp_randint(1, 25),
          'min_samples_split' : sp_randint(2, 25),
          'learning_rate' : sp_uniform(0, 1)}

rsearch_gbr = RandomizedSearchCV(gbr, param_distributions = params, n_jobs = -1,
                                 cv = 5, n_iter = 100, random_state = 0)

rsearch_gbr.fit(X_train_pca, y_train_log)
print(rsearch_gbr.best_params_)

{'learning_rate': 0.11977134909755616, 'max_depth': 12, 'max_features': 49, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 138}


###### Tuned GBR

In [544]:
gbr = GradientBoostingRegressor(**rsearch_gbr.best_params_, random_state = 0)
gbr.fit(X_train_pca, y_train_log)
gbr.score(X_train_pca, y_train_log)

0.9999999997972191

#### XGBR

In [545]:
xgbr.fit(X_train_pca, y_train_log)
xgbr.score(X_train_pca, y_train_log)

0.9999994145798904

#### XGBRFR

In [546]:
xgbrfr.fit(X_train_pca, y_train_log)
xgbrfr.score(X_train_pca, y_train_log)

0.98095888736352

### VotReg

#### VotReg1

In [547]:
votreg.fit(X_train_pca, y_train_log)
votreg.score(X_train_pca, y_train_log)

Learning rate set to 0.028763
0:	learn: 0.6658609	total: 15.9ms	remaining: 15.9s
1:	learn: 0.6565848	total: 25.7ms	remaining: 12.8s
2:	learn: 0.6457129	total: 35.3ms	remaining: 11.7s
3:	learn: 0.6381090	total: 45ms	remaining: 11.2s
4:	learn: 0.6328292	total: 54.8ms	remaining: 10.9s
5:	learn: 0.6261006	total: 64.5ms	remaining: 10.7s
6:	learn: 0.6187049	total: 74.4ms	remaining: 10.6s
7:	learn: 0.6128191	total: 84.1ms	remaining: 10.4s
8:	learn: 0.6023836	total: 93.5ms	remaining: 10.3s
9:	learn: 0.5956610	total: 103ms	remaining: 10.2s
10:	learn: 0.5858189	total: 113ms	remaining: 10.2s
11:	learn: 0.5758920	total: 123ms	remaining: 10.2s
12:	learn: 0.5669337	total: 133ms	remaining: 10.1s
13:	learn: 0.5632598	total: 142ms	remaining: 10s
14:	learn: 0.5565164	total: 152ms	remaining: 9.98s
15:	learn: 0.5480217	total: 162ms	remaining: 9.97s
16:	learn: 0.5420527	total: 172ms	remaining: 9.93s
17:	learn: 0.5342760	total: 181ms	remaining: 9.9s
18:	learn: 0.5241367	total: 192ms	remaining: 9.9s
19:	lear

170:	learn: 0.1250309	total: 2s	remaining: 9.72s
171:	learn: 0.1242820	total: 2.02s	remaining: 9.72s
172:	learn: 0.1237066	total: 2.03s	remaining: 9.72s
173:	learn: 0.1231052	total: 2.05s	remaining: 9.72s
174:	learn: 0.1222375	total: 2.06s	remaining: 9.72s
175:	learn: 0.1213653	total: 2.07s	remaining: 9.7s
176:	learn: 0.1201472	total: 2.08s	remaining: 9.68s
177:	learn: 0.1196019	total: 2.1s	remaining: 9.69s
178:	learn: 0.1188573	total: 2.11s	remaining: 9.69s
179:	learn: 0.1180983	total: 2.13s	remaining: 9.7s
180:	learn: 0.1170873	total: 2.14s	remaining: 9.69s
181:	learn: 0.1164755	total: 2.16s	remaining: 9.69s
182:	learn: 0.1159757	total: 2.17s	remaining: 9.67s
183:	learn: 0.1155865	total: 2.18s	remaining: 9.68s
184:	learn: 0.1151462	total: 2.19s	remaining: 9.67s
185:	learn: 0.1143552	total: 2.21s	remaining: 9.66s
186:	learn: 0.1131836	total: 2.22s	remaining: 9.67s
187:	learn: 0.1126176	total: 2.23s	remaining: 9.65s
188:	learn: 0.1120661	total: 2.25s	remaining: 9.63s
189:	learn: 0.1115

340:	learn: 0.0412459	total: 3.99s	remaining: 7.71s
341:	learn: 0.0409382	total: 4s	remaining: 7.7s
342:	learn: 0.0405856	total: 4.01s	remaining: 7.68s
343:	learn: 0.0403694	total: 4.02s	remaining: 7.67s
344:	learn: 0.0402080	total: 4.03s	remaining: 7.65s
345:	learn: 0.0400699	total: 4.04s	remaining: 7.64s
346:	learn: 0.0399398	total: 4.05s	remaining: 7.63s
347:	learn: 0.0398215	total: 4.06s	remaining: 7.61s
348:	learn: 0.0397600	total: 4.07s	remaining: 7.59s
349:	learn: 0.0396934	total: 4.08s	remaining: 7.58s
350:	learn: 0.0393907	total: 4.09s	remaining: 7.57s
351:	learn: 0.0391993	total: 4.1s	remaining: 7.55s
352:	learn: 0.0388087	total: 4.11s	remaining: 7.53s
353:	learn: 0.0386106	total: 4.12s	remaining: 7.52s
354:	learn: 0.0383577	total: 4.13s	remaining: 7.5s
355:	learn: 0.0382589	total: 4.14s	remaining: 7.49s
356:	learn: 0.0379999	total: 4.15s	remaining: 7.47s
357:	learn: 0.0377167	total: 4.16s	remaining: 7.46s
358:	learn: 0.0375763	total: 4.17s	remaining: 7.45s
359:	learn: 0.0375

499:	learn: 0.0206134	total: 5.62s	remaining: 5.62s
500:	learn: 0.0205788	total: 5.63s	remaining: 5.61s
501:	learn: 0.0205440	total: 5.65s	remaining: 5.6s
502:	learn: 0.0205072	total: 5.65s	remaining: 5.58s
503:	learn: 0.0204862	total: 5.66s	remaining: 5.57s
504:	learn: 0.0203468	total: 5.67s	remaining: 5.56s
505:	learn: 0.0203162	total: 5.68s	remaining: 5.55s
506:	learn: 0.0202182	total: 5.69s	remaining: 5.53s
507:	learn: 0.0201769	total: 5.7s	remaining: 5.52s
508:	learn: 0.0200970	total: 5.71s	remaining: 5.51s
509:	learn: 0.0200371	total: 5.72s	remaining: 5.5s
510:	learn: 0.0200094	total: 5.73s	remaining: 5.48s
511:	learn: 0.0199821	total: 5.74s	remaining: 5.47s
512:	learn: 0.0199618	total: 5.75s	remaining: 5.46s
513:	learn: 0.0199364	total: 5.76s	remaining: 5.45s
514:	learn: 0.0198475	total: 5.77s	remaining: 5.43s
515:	learn: 0.0198249	total: 5.78s	remaining: 5.42s
516:	learn: 0.0198002	total: 5.79s	remaining: 5.41s
517:	learn: 0.0197431	total: 5.8s	remaining: 5.4s
518:	learn: 0.019

665:	learn: 0.0110810	total: 7.62s	remaining: 3.82s
666:	learn: 0.0110131	total: 7.63s	remaining: 3.81s
667:	learn: 0.0109609	total: 7.66s	remaining: 3.81s
668:	learn: 0.0108816	total: 7.68s	remaining: 3.8s
669:	learn: 0.0108307	total: 7.7s	remaining: 3.79s
670:	learn: 0.0108098	total: 7.71s	remaining: 3.78s
671:	learn: 0.0107988	total: 7.72s	remaining: 3.77s
672:	learn: 0.0107883	total: 7.74s	remaining: 3.76s
673:	learn: 0.0107196	total: 7.75s	remaining: 3.75s
674:	learn: 0.0106766	total: 7.76s	remaining: 3.74s
675:	learn: 0.0106095	total: 7.78s	remaining: 3.73s
676:	learn: 0.0106005	total: 7.79s	remaining: 3.72s
677:	learn: 0.0105916	total: 7.81s	remaining: 3.71s
678:	learn: 0.0105167	total: 7.83s	remaining: 3.7s
679:	learn: 0.0104536	total: 7.84s	remaining: 3.69s
680:	learn: 0.0103803	total: 7.86s	remaining: 3.68s
681:	learn: 0.0103174	total: 7.87s	remaining: 3.67s
682:	learn: 0.0102872	total: 7.89s	remaining: 3.66s
683:	learn: 0.0102177	total: 7.91s	remaining: 3.66s
684:	learn: 0.0

840:	learn: 0.0059349	total: 10s	remaining: 1.89s
841:	learn: 0.0058941	total: 10s	remaining: 1.88s
842:	learn: 0.0058877	total: 10s	remaining: 1.87s
843:	learn: 0.0058824	total: 10s	remaining: 1.86s
844:	learn: 0.0058748	total: 10.1s	remaining: 1.84s
845:	learn: 0.0058659	total: 10.1s	remaining: 1.83s
846:	learn: 0.0058391	total: 10.1s	remaining: 1.82s
847:	learn: 0.0058045	total: 10.1s	remaining: 1.81s
848:	learn: 0.0057705	total: 10.1s	remaining: 1.79s
849:	learn: 0.0057274	total: 10.1s	remaining: 1.78s
850:	learn: 0.0057239	total: 10.1s	remaining: 1.77s
851:	learn: 0.0057185	total: 10.1s	remaining: 1.76s
852:	learn: 0.0057124	total: 10.1s	remaining: 1.75s
853:	learn: 0.0056736	total: 10.2s	remaining: 1.74s
854:	learn: 0.0056448	total: 10.2s	remaining: 1.72s
855:	learn: 0.0056236	total: 10.2s	remaining: 1.71s
856:	learn: 0.0056098	total: 10.2s	remaining: 1.7s
857:	learn: 0.0055729	total: 10.2s	remaining: 1.69s
858:	learn: 0.0055668	total: 10.2s	remaining: 1.68s
859:	learn: 0.0055593

0.9941941858997637

#### VotReg2

In [548]:
votreg2.fit(X_train_pca, y_train_log)
votreg2.score(X_train_pca, y_train_log)

0.9843499601411239

### Test Prediction

In [567]:
pred = gbr.predict(X_test_pca)

#### Submission Data 

In [568]:
kaggle = pd.read_csv('Kaggle_Submission_Format.csv')

In [569]:
pred.shape

(36,)

In [570]:
kaggle['Sales(In ThousandDollars)'] = np.exp(pred)

In [571]:
kaggle.to_csv('Iteration_5_pca65_poly2_gbr', index = False)
kaggle.head()

,Year,Sales(In ThousandDollars)
0,1,1072.540987
1,2,627.911843
2,3,2556.377176
3,4,2063.692823
4,5,679.994859
